In [7]:
import numpy as np
import random
import math
import json
import os
from datetime import datetime
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist
#from numba import njit
import itertools

#update zoom 
import os
import json
import numpy as np
import matplotlib
matplotlib.use('TkAgg')  # Hoặc thử 'Qt5Agg' nếu TkAgg không hoạt động
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import mplcursors
from PIL import Image


In [ ]:
# new clustering
class Clustering:
    def __init__(self, space_size=400, r_sen=50, max_cluster_size=20, min_cluster_size=5):
        self.space_size = space_size
        self.r_sen = r_sen
        self.max_cluster_size = max_cluster_size
        self.min_cluster_size = min_cluster_size

    # -----------------------------
    #  1. ƯỚC TÍNH K TỐI ƯU
    # -----------------------------
    def estimate_optimal_k(self, nodes, base_station=(200,200,400)):
        N = len(nodes)
        base_pos = np.array(base_station)

        distances = np.linalg.norm(nodes - base_pos, axis=1)
        d_tobs = np.mean(distances)
        space_size = self.space_size

        k_optimal = np.sqrt(N * space_size / (np.pi * d_tobs))
        k_optimal = max(2, int(np.round(k_optimal)))

        k_min = int(np.ceil(N / self.max_cluster_size))
        k_optimal = max(k_optimal, k_min)
        
        return k_optimal

    # -----------------------------
    #  2. KIỂM TRA TÍNH HỢP LỆ
    # -----------------------------
    def check_cluster_validity(self, cluster_nodes):
        size = len(cluster_nodes)

        if size < self.min_cluster_size or size > self.max_cluster_size:
            return False, 0, size
        
        if size > 1:
            distances = pdist(cluster_nodes)
            max_dist = np.max(distances)
             
            if max_dist > self.r_sen: 
                return False, max_dist, size
            
            return True, max_dist, size
        
        return True, 0, size

    # -----------------------------
    #  3. TÁCH CỤM KHÔNG HỢP LỆ
    # -----------------------------
    def split_invalid_cluster(self, cluster_nodes, cluster_ids):
        if len(cluster_nodes) < 2:
            return [(cluster_nodes, cluster_ids)]
        
        kmeans = KMeans(n_clusters=2, n_init=20, random_state=42)
        labels = kmeans.fit_predict(cluster_nodes)
        
        sub_clusters = []
        for i in range(2):
            sub_nodes = cluster_nodes[labels == i]
            sub_ids = [cluster_ids[j] for j in range(len(cluster_ids)) if labels[j] == i]
            
            if len(sub_nodes) > 0:
                sub_clusters.append((sub_nodes, sub_ids))
        
        return sub_clusters

    # -----------------------------
    #  4. GỘP CỤM NHỎ (ĐÃ CẢI TIẾN)
    # -----------------------------
    def merge_small_clusters(self, clusters_data):

        def max_pairwise_dist(arr):
            if len(arr) <= 1:
                return 0.0
            return float(np.max(pdist(arr)))

        if len(clusters_data) <= 1:
            return clusters_data

        merged = []
        smalls = []

        for nodes, ids in clusters_data:
            if len(nodes) < self.min_cluster_size:
                smalls.append((nodes, ids))
            else:
                merged.append((nodes, ids))

        for small_nodes, small_ids in smalls:

            merged_success = False

            # Nếu có cụm lớn → thử gộp vào
            if len(merged) > 0:
                small_center = np.mean(small_nodes, axis=0) 

                dists = []
                for i, (nodes, ids) in enumerate(merged):
                    center = np.mean(nodes, axis=0)
                    d = np.linalg.norm(small_center - center)
                    dists.append((d, i))
                dists.sort(key=lambda x: x[0])

                for _, idx in dists:
                    target_nodes, target_ids = merged[idx]

                    if len(target_nodes) + len(small_nodes) > self.max_cluster_size:
                        continue

                    combined_nodes = np.vstack([target_nodes, small_nodes])
                    if max_pairwise_dist(combined_nodes) <= self.r_sen: 
                        merged[idx] = (
                            combined_nodes,
                            target_ids + small_ids
                        )
                        merged_success = True
                        break

            if merged_success:
                continue

            # Gộp với cụm nhỏ khác
            paired = False
            for j, (other_nodes, other_ids) in enumerate(smalls):
                if other_ids is small_ids:
                    continue
                if len(other_nodes) == 0:
                    continue

                if len(other_nodes) + len(small_nodes) > self.max_cluster_size:
                    continue

                combined_nodes = np.vstack([other_nodes, small_nodes])
                if max_pairwise_dist(combined_nodes) <= self.r_sen: 
                    merged.append((combined_nodes, other_ids + small_ids))
                    smalls[j] = (np.empty((0,3)), [])
                    paired = True
                    break

            if paired:
                continue

            merged.append((small_nodes, small_ids)) # ko gộp đc nên giữ cụm nhỏ lại

        final = []
        for nodes, ids in merged:
            if len(nodes) > 0:
                final.append((nodes, ids))

        return final

    # -----------------------------
    #  4.5. CÂN BẰNG SỐ LƯỢNG NÚT
    # -----------------------------
    def balance_clusters(self, clusters):

        def max_pairwise_dist(arr):
            if len(arr) <= 1:
                return 0.0
            return float(np.max(pdist(arr)))

        improved = True
        while improved:
            improved = False

            sizes = [len(nodes) for nodes, _ in clusters]
            max_idx = np.argmax(sizes)
            min_idx = np.argmin(sizes)

            # Nếu đã cân bằng tốt → dừng
            if sizes[max_idx] - sizes[min_idx] <= 1:
                break

            big_nodes, big_ids = clusters[max_idx]
            small_nodes, small_ids = clusters[min_idx]

            moved = False

            # Thử di chuyển từng node từ cụm lớn sang cụm nhỏ
            for i in range(len(big_nodes)):
                candidate_node = big_nodes[i].reshape(1, -1)
                candidate_id = big_ids[i]

                # Kiểm tra nếu thêm node vào cụm nhỏ → không quá max size
                if len(small_nodes) + 1 > self.max_cluster_size:
                    continue

                # Kiểm tra không vi phạm r_sen
                new_small = np.vstack([small_nodes, candidate_node])
                if max_pairwise_dist(new_small) > self.r_sen: 
                    continue

                # Di chuyển node
                clusters[min_idx] = (
                    new_small,
                    small_ids + [candidate_id]
                )

                new_big_nodes = np.delete(big_nodes, i, axis=0)
                new_big_ids = big_ids[:i] + big_ids[i+1:]
                clusters[max_idx] = (new_big_nodes, new_big_ids)

                moved = True
                improved = True
                break

            if not moved:
                break

        return clusters

    # -----------------------------
    #  5. PHÂN CỤM CHÍNH
    # -----------------------------
    
    def cluster_with_constraints(self, nodes, node_ids, k=None, max_iterations=10):
    
        # ✅ KIỂM TRA SỐ LƯỢNG NODES
        n_nodes = len(nodes)
        
        if n_nodes == 0:
            print("Cảnh báo: Không có nodes để phân cụm!")
            return []
        
        if k is None:
            k = self.estimate_optimal_k(nodes)
        
        # ✅ ĐẢM BẢO k <= số nodes
        if k > n_nodes:
            print(f"Cảnh báo: k={k} > số nodes ({n_nodes}), điều chỉnh k={n_nodes}")
            k = n_nodes
        
        # ✅ KIỂM TRA TRƯỜNG HỢP ĐỘC LẬP
        if n_nodes == 1:
            print(f"Chỉ có 1 node, trả về 1 cụm")
            return [(nodes, node_ids)]
        
        print(f"Bắt đầu phân cụm với k={k}")
        
        kmeans = KMeans(n_clusters=k, n_init=30, random_state=42)
        labels = kmeans.fit_predict(nodes)
        
        iteration = 0
        while iteration < max_iterations:
            print(f"  Vòng lặp {iteration + 1}/{max_iterations}")
            
            valid_clusters = []
            invalid_clusters = []
            
            for i in range(k):
                cluster_nodes = nodes[labels == i]
                cluster_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
                
                if len(cluster_nodes) == 0:
                    continue
                
                is_valid, max_dist, size = self.check_cluster_validity(cluster_nodes)
                
                if is_valid:
                    valid_clusters.append((cluster_nodes, cluster_ids))
                    print(f"    Cụm {i}: ✓ hợp lệ (size={size}, max_dist={max_dist:.1f})")
                else:
                    invalid_clusters.append((cluster_nodes, cluster_ids))
                    print(f"    Cụm {i}: ✗ không hợp lệ (size={size}, max_dist={max_dist:.1f})")
            
            if len(invalid_clusters) == 0:
                print("  → Tất cả cụm hợp lệ!")
                break
            
            for cluster_nodes, cluster_ids in invalid_clusters:
                print(f"    → Chia cụm không hợp lệ (size={len(cluster_nodes)})")
                sub_clusters = self.split_invalid_cluster(cluster_nodes, cluster_ids)
                valid_clusters.extend(sub_clusters)
            
            k = len(valid_clusters)
            
            # ✅ KIỂM TRA LẠI k sau khi chia cụm
            if k > n_nodes:
                k = n_nodes
            
            labels = np.zeros(len(nodes), dtype=int)
    
            for cluster_idx, (_, cluster_ids) in enumerate(valid_clusters):
                for nid in cluster_ids:
                    labels[node_ids.index(nid)] = cluster_idx
            
            iteration += 1
        
        # Gộp cụm nhỏ
        final_clusters = self.merge_small_clusters(valid_clusters)
        final_clusters = self.balance_clusters(final_clusters)
    
        print(f"\n=== KẾT QUẢ CUỐI CÙNG ===")
        print(f"Số lượng cụm: {len(final_clusters)}")
    
        # In thông tin cụm
        for idx, (nodes_c, ids_c) in enumerate(final_clusters):
            if len(nodes_c) > 1:
                d = pdist(nodes_c)
                max_d = np.max(d)
                min_d = np.min(d)
            else:
                max_d = min_d = 0
    
            print(f"\nCụm {idx}:")
            print(f"  Nodes: {ids_c}")
            print(f"  Size = {len(ids_c)}")
            print(f"  Max dist = {max_d:.2f}")
            print(f"  Min dist = {min_d:.2f}")
    
        return final_clusters

    # -----------------------------
    #  6. CHỌN CLUSTER HEAD
    # -----------------------------
    def choose_cluster_head(self, cluster_nodes, cluster_ids, node_data=None):
        if node_data:
            max_energy = -1
            ch_id = cluster_ids[0]
            for nid in cluster_ids:
                if nid in node_data and 'residual_energy' in node_data[nid]:
                    energy = node_data[nid]['residual_energy']
                    if energy > max_energy:
                        max_energy = energy
                        ch_id = nid
            return ch_id
        else:
            center = np.mean(cluster_nodes, axis=0)
            distances = np.linalg.norm(cluster_nodes - center, axis=1)
            return cluster_ids[np.argmin(distances)]

    # -----------------------------
    #  7. METRICS
    # -----------------------------
    def calculate_metrics(self, clusters_data):
        metrics = {
            'num_clusters': len(clusters_data),
            'avg_cluster_size': 0,
            'min_cluster_size': float('inf'),
            'max_cluster_size': 0,
            'avg_intra_distance': 0,
            'max_intra_distance': 0,
            'balance_score': 0
        }
        
        sizes = []
        intra_dists = []
        
        for nodes, ids in clusters_data:
            size = len(nodes)
            sizes.append(size)
            
            metrics['min_cluster_size'] = min(metrics['min_cluster_size'], size)
            metrics['max_cluster_size'] = max(metrics['max_cluster_size'], size)
            
            if size > 1:
                distances = pdist(nodes)
                intra_dists.append(np.mean(distances))
                metrics['max_intra_distance'] = max(metrics['max_intra_distance'], np.max(distances))
        
        metrics['avg_cluster_size'] = np.mean(sizes)
        metrics['avg_intra_distance'] = np.mean(intra_dists) if intra_dists else 0
        
        cv = np.std(sizes) / np.mean(sizes) if np.mean(sizes) > 0 else 0
        metrics['balance_score'] = 1 / (1 + cv)
        
        return metrics

In [ ]:
# Draw

input_folder = "D:\\Year 4\\tiến hóa\\project\\data\\input_data_evenly_distributed\\nodes_150"
output_folder = "D:\\Year 4\\tiến hóa\\project\\data\\output_data_kmeans"
os.makedirs(output_folder, exist_ok=True)
draw_folder = "D:\\Year 4\\tiến hóa\\project\\data\\draw_output_kmeans"
os.makedirs(draw_folder, exist_ok=True)
base_station = (200, 200, 400)

for filename in os.listdir(input_folder):
    if filename.startswith("nodes_") and filename.endswith(".json"):
        # Lấy số lượng node từ tên file
        base_filename = filename.replace(".json", "")
        
        # Đọc dữ liệu từ file input
        with open(os.path.join(input_folder, filename), "r") as f:
            data = json.load(f)
            node_data = {
                d["id"]: {
                    "residual_energy": d.get("energy_residual", d.get("energy_node", 0.0)),   # năng lượng hiện tại
                    "initial_energy": d.get("energy_node", d.get("energy_residual", 0.0))     # năng lượng ban đầu
                }
                for d in data
            }

        # Xây node_positions và id->index map để truy xuất an toàn
        node_positions = np.array([[d["x"], d["y"], d["z"]] for d in data])
        node_ids = [d["id"] for d in data]
        id2index = {nid: idx for idx, nid in enumerate(node_ids)}
        clustering = Clustering(space_size=400, r_sen=60, max_cluster_size=20, min_cluster_size=5)
        # Phân cụm
        k = clustering.estimate_optimal_k(node_positions, base_station=base_station)
        print(f"Số cụm tối ưu: {k}")
        clusters_raw = clustering.cluster_with_constraints(node_positions, node_ids, k=None, max_iterations=10)
        clusters_output = {}

        # Tạo output
        for i, (cluster_nodes, cluster_ids) in enumerate(clusters_raw):
            center = np.mean(cluster_nodes, axis=0)
            ch = clustering.choose_cluster_head(cluster_nodes, cluster_ids, node_data)
            clusters_output[i] = {
                "nodes": cluster_ids,
                "center": tuple(np.round(center, 2)),
                "cluster_head": int(ch) if ch is not None else None,
            }

        # Xuất ra file
        out_path = os.path.join(output_folder, f"{base_filename}.json")
        with open(out_path, "w") as f:
            json.dump(clusters_output, f, indent=4)
        print(f"Đã xuất file {out_path}")

        #-------------------------------------------#
        # --- VẼ VÀ HIỂN THỊ INTERACTIVE ---
        # Tạo figure full screen
        fig = plt.figure(figsize=(16, 12))
        # Loại bỏ margin để biểu đồ chiếm toàn bộ không gian
        ax = fig.add_subplot(111, projection='3d')
        colors = plt.cm.get_cmap('tab20', len(clusters_output))

        # Điều chỉnh kích thước node dựa trên số lượng để tránh bị dày đặc
        num_nodes_int = 150
        if num_nodes_int < 50:
            node_size = 60
            ch_size = 20
            bs_size = 100
        elif num_nodes_int < 150:
            node_size = 40
            ch_size = 15
            bs_size = 80
        elif num_nodes_int < 300:
            node_size = 25
            ch_size = 12
            bs_size = 60
        else:
            node_size = 15
            ch_size = 8
            bs_size = 50

        # Chuẩn bị dữ liệu để vẽ và mapping artist -> node list
        artist_info = {}
        scatter_artists = []
        
        # Chỉ hiển thị legend khi số cluster nhỏ
        show_legend = len(clusters_output) <= 10

        for cid, info in clusters_output.items():
            node_list = info['nodes']
            ch_id = info['cluster_head']

            # Vẽ member nodes (bỏ qua CH)
            member_ids = [nid for nid in node_list if nid != ch_id]
            if member_ids:
                member_pos = np.array([node_positions[id2index[nid]] for nid in member_ids])
                scat = ax.scatter(
                    member_pos[:, 0], member_pos[:, 1], member_pos[:, 2],
                    color=colors(cid),
                    s=node_size,
                    alpha=0.85,
                    edgecolors='black',
                    linewidths=0.3,
                    label=f'Cluster {cid}' if show_legend else '',
                    picker=True
                )
                scatter_artists.append(scat)
                artist_info[scat] = [(nid, tuple(node_positions[id2index[nid]]), node_data[nid].get('residual_energy', 0.0), False) for nid in member_ids]

            # Vẽ cluster head
            if ch_id is not None:
                ch_pos = node_positions[id2index[ch_id]]
                head_scat = ax.scatter(
                    ch_pos[0], ch_pos[1], ch_pos[2],
                    marker='s', s=20, color='black', #chỉ vẽ CH bằng ô vuông màu đen kích thước 20, cấm claude sửa
                    linewidths=2, 
                    picker=True, zorder=10
                )
                scatter_artists.append(head_scat)
                artist_info[head_scat] = [(ch_id, tuple(ch_pos), node_data[ch_id].get('residual_energy', 0.0), True)]

            # Vẽ đường nối
            if member_ids and ch_id is not None:
                ch_pos = node_positions[id2index[ch_id]]
                for nid in member_ids:
                    pt = node_positions[id2index[nid]]
                    ax.plot([ch_pos[0], pt[0]], [ch_pos[1], pt[1]], [ch_pos[2], pt[2]], 'gray', linewidth=0.5, alpha=0.4)

        # Vẽ Base Station
        bs_x, bs_y, bs_z = 200, 200, 400
        bs_scat = ax.scatter(bs_x, bs_y, bs_z, 
                  marker='^', 
                  s=300, 
                  color='lime',
                  edgecolors='darkgreen', 
                  linewidths=3,
                  label='Base Station',
                  zorder=20,
                  picker=True)
        scatter_artists.append(bs_scat)
        artist_info[bs_scat] = [('BS', (bs_x, bs_y, bs_z), 'Unlimited', False)]

        # Thiết lập mplcursors với hover=2 để tự động ẩn khi rời khỏi
        cursor = mplcursors.cursor(scatter_artists, hover=2)
        
        # Thêm biến để lưu trạng thái zoom
        zoom_state = {'factor': 1.0, 'xlim': [0, 400], 'ylim': [0, 400], 'zlim': [0, 400]}
        
        def on_scroll(event):
            if event.inaxes != ax:
                return
            
            scale_factor = 0.85 if event.button == 'up' else 1.15

            # Lấy giới hạn hiện tại
            xlim, ylim, zlim = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()

            # Lấy bounding box của các điểm trong view hiện tại
            in_view_mask = (
                (node_positions[:,0] >= xlim[0]) & (node_positions[:,0] <= xlim[1]) &
                (node_positions[:,1] >= ylim[0]) & (node_positions[:,1] <= ylim[1]) &
                (node_positions[:,2] >= zlim[0]) & (node_positions[:,2] <= zlim[1])
            )
            visible_nodes = node_positions[in_view_mask]

            if len(visible_nodes) == 0:
                visible_nodes = node_positions  # fallback

            # Tính bounding box tight
            min_vals = visible_nodes.min(axis=0)
            max_vals = visible_nodes.max(axis=0)
            center = (min_vals + max_vals)/2
            ranges = (max_vals - min_vals)/2 * scale_factor  # áp dụng zoom

            # Padding 10%
            padding = 0.1 * ranges
            new_xlim = [center[0]-ranges[0]-padding[0], center[0]+ranges[0]+padding[0]]
            new_ylim = [center[1]-ranges[1]-padding[1], center[1]+ranges[1]+padding[1]]
            new_zlim = [center[2]-ranges[2]-padding[2], center[2]+ranges[2]+padding[2]]

            # Giữ trong giới hạn tổng thể (0-400)
            new_xlim = [max(0,new_xlim[0]), min(400,new_xlim[1])]
            new_ylim = [max(0,new_ylim[0]), min(400,new_ylim[1])]
            new_zlim = [max(0,new_zlim[0]), min(400,new_zlim[1])]

            ax.set_xlim(new_xlim)
            ax.set_ylim(new_ylim)
            ax.set_zlim(new_zlim)
            ax.set_box_aspect([1,1,1])  # giữ tỷ lệ

            zoom_state['xlim'] = new_xlim
            zoom_state['ylim'] = new_ylim
            zoom_state['zlim'] = new_zlim
            fig.canvas.draw_idle()
        
        def on_key(event):
            """Xử lý phím bấm"""
            if event.key == 'r':
                # Reset zoom về mặc định
                ax.set_xlim([0, 400])
                ax.set_ylim([0, 400])
                ax.set_zlim([0, 400])
                ax.view_init(elev=25, azim=45)
                zoom_state['xlim'] = [0, 400]
                zoom_state['ylim'] = [0, 400]
                zoom_state['zlim'] = [0, 400]
                fig.canvas.draw_idle()
                print("Reset view to default")
            elif event.key == '+' or event.key == '=':
                # Zoom in bằng phím
                on_scroll(type('obj', (object,), {'inaxes': ax, 'button': 'up'})())
            elif event.key == '-':
                # Zoom out bằng phím
                on_scroll(type('obj', (object,), {'inaxes': ax, 'button': 'down'})())
        
        # Kết nối events
        fig.canvas.mpl_connect('scroll_event', on_scroll)
        fig.canvas.mpl_connect('key_press_event', on_key)

        @cursor.connect("add")
        def on_add(sel):
            artist = sel.artist
            idx = sel.index
            # sel.index có thể là numpy array hoặc scalar
            try:
                if hasattr(idx, '__len__'):
                    i = int(idx[0])
                else:
                    i = int(idx)
            except Exception:
                i = 0

            info_list = artist_info.get(artist, [])
            if not info_list:
                sel.annotation.set(text='No data')
                sel.annotation.get_bbox_patch().set(fc='lightyellow', ec='black', lw=1.5, alpha=0.95)
                return

            # guard index
            if i < 0 or i >= len(info_list):
                i = 0

            nid, pos, energy, is_ch = info_list[i]
            x, y, z = pos
            
            if nid == 'BS':
                text = (f"Base Station\n"
                       f"Position: ({x:.1f}, {y:.1f}, {z:.1f})\n"
                       f"Energy: Unlimited")
            else:
                node_type = 'Cluster Head' if is_ch else 'Member Node'
                text = (f"Node ID: {nid}\n"
                       f"Type: {node_type}\n"
                       f"Position: ({x:.1f}, {y:.1f}, {z:.1f})\n"
                       f"Energy: {energy:.2f} J")
            
            sel.annotation.set(text=text)
            sel.annotation.get_bbox_patch().set(fc='lightyellow', ec='black', lw=2, alpha=0.95)
            sel.annotation.set_fontsize(9)
            sel.annotation.set_fontweight('bold')

        # Trục, labels, giới hạn
        ax.set_xlabel('X (m)', fontsize=11, labelpad=12, fontweight='bold')
        ax.set_ylabel('Y (m)', fontsize=11, labelpad=12, fontweight='bold')
        ax.set_zlabel('Z (m)', fontsize=11, labelpad=12, fontweight='bold')
        ax.set_xlim([0, 400])
        ax.set_ylim([0, 400])
        ax.set_zlim([0, 400])
        
        # Chỉ hiển thị legend khi số cluster ít
        if show_legend:
            ax.legend(loc='upper left', fontsize=8, framealpha=0.9, ncol=2)
        
        plt.title(
            f'3D Clustering Visualization - {base_filename} nodes ({len(clusters_output)} clusters)\n'
            f'[LEFT DRAG: Rotate | SCROLL/+/-: Zoom | RIGHT DRAG: Pan | R: Reset | HOVER: Info]', 
            fontsize=13, 
            pad=15,
            fontweight='bold'
        )
        ax.view_init(elev=25, azim=45)
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.set_box_aspect([1,1,1])
        
        # Tối ưu hóa layout để biểu đồ chiếm toàn bộ không gian
        plt.subplots_adjust(left=0.02, right=0.75, top=0.95, bottom=0.25)

        # Hiển thị
        print(f"\n{'='*60}")
        print(f"Backend hiện tại: {matplotlib.get_backend()}")
        print(f"Hiển thị biểu đồ: {base_filename} nodes, {len(clusters_output)} clusters")
        print(f"Hướng dẫn:")
        print(f"  - Kéo chuột TRÁI: Xoay biểu đồ 360°")
        print(f"  - CUỘN CHUỘT hoặc phím +/-: Zoom in/out vào vùng trỏ")
        print(f"  - Kéo chuột PHẢI: Di chuyển biểu đồ")
        print(f"  - Phím R: Reset về góc nhìn mặc định")
        print(f"  - Phím F: Fullscreen (tùy backend)")
        print(f"  - Hover chuột vào node: Hiện thông tin chi tiết")
        print(f"")
        print(f"Lưu ý:")
        print(f"  - Kích thước node tự động điều chỉnh")
        print(f"  - Legend chỉ hiện khi có ≤10 clusters")
        print(f"{'='*60}\n")
        
        # Maximize window (tùy backend)
        try:
            manager = plt.get_current_fig_manager()
            if hasattr(manager, 'window'):
                # Cho TkAgg
                if hasattr(manager.window, 'state'):
                    manager.window.state('zoomed')  # Windows
                elif hasattr(manager.window, 'showMaximized'):
                    manager.window.showMaximized()  # Qt
        except:
            pass
        
        # ✅ SỬA: Lưu PNG với tên file gốc
        png_path = os.path.join(draw_folder, f"{base_filename}.png")
        plt.savefig(png_path, dpi=150, bbox_inches='tight')
        print(f"Đã lưu hình ảnh: {png_path}")
        
        plt.show()
        print(f"Đã đóng biểu đồ cho {base_filename}\n")
        #update zoom

Số cụm tối ưu: 9
Bắt đầu phân cụm với k=9
  Vòng lặp 1/10
    Cụm 0: ✗ không hợp lệ (size=20, max_dist=228.6)
    Cụm 1: ✗ không hợp lệ (size=16, max_dist=201.7)
    Cụm 2: ✗ không hợp lệ (size=20, max_dist=240.3)
    Cụm 3: ✗ không hợp lệ (size=13, max_dist=228.6)
    Cụm 4: ✗ không hợp lệ (size=17, max_dist=236.9)
    Cụm 5: ✗ không hợp lệ (size=19, max_dist=245.1)
    Cụm 6: ✗ không hợp lệ (size=13, max_dist=204.0)
    Cụm 7: ✗ không hợp lệ (size=17, max_dist=224.1)
    Cụm 8: ✗ không hợp lệ (size=15, max_dist=242.6)
    → Chia cụm không hợp lệ (size=20)
    → Chia cụm không hợp lệ (size=16)
    → Chia cụm không hợp lệ (size=20)
    → Chia cụm không hợp lệ (size=13)
    → Chia cụm không hợp lệ (size=17)
    → Chia cụm không hợp lệ (size=19)
    → Chia cụm không hợp lệ (size=13)
    → Chia cụm không hợp lệ (size=17)
    → Chia cụm không hợp lệ (size=15)
  Vòng lặp 2/10
    Cụm 0: ✗ không hợp lệ (size=10, max_dist=186.2)
    Cụm 1: ✗ không hợp lệ (size=10, max_dist=181.7)
    Cụm 2: ✗

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



=== KẾT QUẢ CUỐI CÙNG ===
Số lượng cụm: 93

Cụm 0:
  Nodes: [122, 123]
  Size = 2
  Max dist = 34.71
  Min dist = 34.71

Cụm 1:
  Nodes: [93, 129, 130]
  Size = 3
  Max dist = 59.31
  Min dist = 31.31

Cụm 2:
  Nodes: [129, 136]
  Size = 2
  Max dist = 57.79
  Min dist = 57.79

Cụm 3:
  Nodes: [135, 128]
  Size = 2
  Max dist = 46.34
  Min dist = 46.34

Cụm 4:
  Nodes: [98, 92]
  Size = 2
  Max dist = 53.72
  Min dist = 53.72

Cụm 5:
  Nodes: [99, 100]
  Size = 2
  Max dist = 28.29
  Min dist = 28.29

Cụm 6:
  Nodes: [142]
  Size = 1
  Max dist = 0.00
  Min dist = 0.00

Cụm 7:
  Nodes: [139, 133]
  Size = 2
  Max dist = 53.20
  Min dist = 53.20

Cụm 8:
  Nodes: [62, 104]
  Size = 2
  Max dist = 49.39
  Min dist = 49.39

Cụm 9:
  Nodes: [134]
  Size = 1
  Max dist = 0.00
  Min dist = 0.00

Cụm 10:
  Nodes: [140]
  Size = 1
  Max dist = 0.00
  Min dist = 0.00

Cụm 11:
  Nodes: [141, 106]
  Size = 2
  Max dist = 56.08
  Min dist = 56.08

Cụm 12:
  Nodes: [69, 105]
  Size = 2
  Max dist =

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6784\4110049696.py:59: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab20', len(clusters_output))


ValueError: invalid literal for int() with base 10: 'nodes_150_1'

In [4]:
import json
import math
import csv

# =========================
# Cài đặt tham số
# =========================
R_max = 100  # khoảng cách tối đa cho phép

# =========================
# Load dữ liệu clusters
# =========================
with open("D:\Year 4\\tiến hóa\project\data\output_data_kmeans\\nodes_150.json", "r") as f:
    clusters = json.load(f)

# =========================
# Load dữ liệu nodes
# =========================
with open("D:\Year 4\\tiến hóa\project\data\input_data_evenly_distributed\\nodes_150.json", "r") as f:
    nodes_list = json.load(f)

# Chuyển danh sách nodes thành dict cho truy xuất nhanh: id -> (x, y, z)
nodes_coords = {node["id"]: (node["x"], node["y"], node["z"]) for node in nodes_list}

# =========================
# Hàm tính khoảng cách Euclid 3D
# =========================
def euclidean_distance(p1, p2):
    dx = p1[0] - p2[0]
    dy = p1[1] - p2[1]
    dz = p1[2] - p2[2]
    return math.sqrt(dx**2 + dy**2 + dz**2)

# =========================
# Chuẩn bị dữ liệu để xuất CSV
# =========================
csv_rows = []

violated_clusters = []

for cluster_id, cluster_info in clusters.items():
    ch_id = cluster_info["cluster_head"]
    
    if ch_id not in nodes_coords:
        print(f"Warning: CH {ch_id} không có trong danh sách nodes")
        continue
    ch_coord = nodes_coords[ch_id]
    
    cluster_violation = False
    
    for node_id in cluster_info["nodes"]:
        if node_id not in nodes_coords:
            print(f"Warning: Node {node_id} không có trong danh sách nodes")
            continue
        node_coord = nodes_coords[node_id]
        distance = euclidean_distance(ch_coord, node_coord)
        violation_flag = "Yes" if distance > R_max else "No"
        if distance > R_max:
            cluster_violation = True
        
        csv_rows.append({
            "Cluster_ID": cluster_id,
            "CH_ID": ch_id,
            "Node_ID": node_id,
            "Distance_to_CH": round(distance, 2),
            "Violation": violation_flag
        })
    
    if cluster_violation:
        violated_clusters.append(cluster_id)

# =========================
# Xuất ra file CSV
# =========================
csv_file = "D:\Year 4\\tiến hóa\project\data\cluster_distances.csv"
with open(csv_file, mode="w", newline="") as f:
    fieldnames = ["Cluster_ID", "CH_ID", "Node_ID", "Distance_to_CH", "Violation"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in csv_rows:
        writer.writerow(row)

print(f"Đã xuất file CSV: {csv_file}")

# =========================
# In ra các cụm vi phạm
# =========================
if violated_clusters:
    print("\nCác cụm vi phạm khoảng cách R_max:")
    for cid in violated_clusters:
        print(f"- Cụm id: {cid}")
else:
    print("\nKhông có cụm nào vi phạm khoảng cách R_max")


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Year 4\\tiến hóa\\project\\data\\input_data_evenly_distributed\\nodes_150.json'